In [1]:
import numpy as np
import pandas as pd
"""
Series: (index, scalar value) pairs
DataFrame: (column, Series) pairs
Panel: (item, DataFrame) pairs
"""

'\nSeries: (index, scalar value) pairs\nDataFrame: (column, Series) pairs\nPanel: (item, DataFrame) pairs\n'

### value is NaN when value not assigned by index.

In [2]:
df = pd.DataFrame({'one' : pd.Series(np.arange(3), index=['a', 'b', 'c']),
                   'two' : pd.Series(np.arange(4)+3, index=['a', 'b', 'c', 'd']),
                   'three' : pd.Series(np.arange(3)+6, index=['b', 'c', 'd'])}); df

,one,two,three
a,0.0,3,NaN
b,1.0,4,6.0
c,2.0,5,7.0
d,NaN,6,8.0


In [3]:
df_fillna = df.copy()
Default = 0
df_fillna[df.isna()] = Default
df_fillna

,one,two,three
a,0.0,3,0.0
b,1.0,4,6.0
c,2.0,5,7.0
d,0.0,6,8.0


In [4]:
row, column = df.iloc[1], df['two'] ;row

one      1.0
two      4.0
three    6.0
Name: b, dtype: float64

### .loc() for label slicing.

In [5]:
columnaw = df.loc[:,'two']; columnaw

a    3
b    4
c    5
d    6
Name: two, dtype: int64

In [6]:
df.sub(row, axis='columns')

,one,two,three
a,-1.0,-1.0,NaN
b,0.0,0.0,0.0
c,1.0,1.0,1.0
d,NaN,2.0,2.0


In [7]:
df.sub(column, axis='index')

,one,two,three
a,-3.0,0,NaN
b,-3.0,0,2.0
c,-3.0,0,2.0
d,NaN,0,2.0


#### axis 0 -> index, asix 1 -> column

In [8]:
dfmi = df.copy()
dfmi.index = pd.MultiIndex.from_tuples([(1,'a'),(1,'b'),(1,'c'),(2,'a')], names = ['first','second'])
dfmi.sub(column, axis=0, level='second')

one  two  three
first second                 
1     a      -3.0    0    NaN
      b      -3.0    0    2.0
      c      -3.0    0    2.0
2     a       NaN    3    5.0

In [9]:
s = pd.Series(np.arange(10)); 
div, rem = divmod(s, 3); 
rem #idx = pd.Index(np.arange(10)) ok as well

0    0
1    1
2    2
3    0
4    1
5    2
6    0
7    1
8    2
9    0
dtype: int64

In [10]:
pd.Index(np.arange(10))

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

In [42]:
pd.Index(range(10))

RangeIndex(start=0, stop=10, step=1)

In [43]:
d, r = divmod(pd.Index(range(10)), 3); r

Int64Index([0, 1, 2, 0, 1, 2, 0, 1, 2, 0], dtype='int64')

In [12]:
#elementwise
div, rem = divmod(s, [2, 2, 3, 3, 4, 4, 5, 5, 6, 6]) 

In [13]:
df = pd.DataFrame({'one' : pd.Series(np.arange(3), index=['a', 'b', 'c']),
                   'two' : pd.Series(np.arange(4)+3, index=['a', 'b', 'c', 'd']),
                   'three' : pd.Series(np.arange(3)+6, index=['b', 'c', 'd'])}); df

,one,two,three
a,0.0,3,NaN
b,1.0,4,6.0
c,2.0,5,7.0
d,NaN,6,8.0


In [14]:
df.sum(axis=1, skipna=True)

a     3.0
b    11.0
c    14.0
d    14.0
dtype: float64

In [44]:
df + df == df*2

,one,two,three
a,True,True,False
b,True,True,True
c,True,True,True
d,False,True,True


In [47]:
# all() looks for column wise when DataFrame
(df + df == df*2).all()

one      False
two       True
three    False
dtype: bool

In [15]:
(df + df == df*2).all().all()

False

In [16]:
(df+df).equals(df*2)

True

In [17]:
df1 = pd.DataFrame({'A' : [1., np.nan, 3., 5., np.nan], 'B' : [np.nan, 2., 3., np.nan, 6.]})
df2 = pd.DataFrame({'A' : [5., 2., 4., np.nan, 3., 7.], 'B' : [np.nan, np.nan, 3., 4., 6., 8.]})
df1.combine_first(df2)

,A,B
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0


In [18]:
combiner = lambda x, y: np.where(pd.isna(x), y, x)
df1.combine(df2, combiner)

,A,B
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0


In [19]:
#cum production
ddf = pd.DataFrame(index=pd.date_range('2008-11-21', periods=3, freq='3D'), data = {'Return':np.random.randn(3)})
ddf.ix["Cumulative"] = (ddf['Return']+1).prod() - 1
#ddf.ix["Cumulative"] = ((ddf+1).cumprod()-1).iloc[-1]
ddf

/Users/ryota/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


,Return
2008-11-21 00:00:00,-0.324191
2008-11-24 00:00:00,1.521671
2008-11-27 00:00:00,-1.381697
Cumulative,-1.650476


In [20]:
ts_stand = (df - df.mean()) / df.std(); ts_stand.std()
(df - df.mean()).pipe(lambda x: x/ df.std()).pipe(lambda y: y.std())

one      1.0
two      1.0
three    1.0
dtype: float64

In [21]:
series = pd.Series(np.random.randn(1000))
series.nunique()
series.describe(percentiles=[.05, .25, .75, .95])

count    1000.000000
mean        0.069676
std         1.038959
min        -2.623824
5%         -1.558610
25%        -0.684572
50%         0.037215
75%         0.793995
95%         1.845615
max         3.634131
dtype: float64

In [22]:
frame = pd.DataFrame({'a': ['Yes', 'Yes', 'No', 'No', 'No Responce'], 'b': range(5)})
frame.describe()

,b
count,5.000000
mean,2.000000
std,1.581139
min,0.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,4.000000


In [23]:
frame.describe(include=['object'])
frame.describe(include=['number'])
frame.describe(include='all')

,a,b
count,5,5.000000
unique,3,NaN
top,No,NaN
freq,2,NaN
mean,NaN,2.000000
std,NaN,1.581139
min,NaN,0.000000
25%,NaN,1.000000
50%,NaN,2.000000
75%,NaN,3.000000


In [24]:
df_ = pd.DataFrame([list(map(lambda x:x+i, list(range(3)))) for i in range(10)], columns=['A', 'B', 'C'],index=pd.date_range('1/1/2000', periods=10))
tsdf = df_.copy()
from random import shuffle
shuffle(tsdf['A'].values); shuffle(tsdf['B'].values),shuffle(tsdf['C'].values)
tsdf.apply(lambda s:[s.idxmax(),s.idxmin()])

A    [2000-01-08 00:00:00, 2000-01-03 00:00:00]
B    [2000-01-08 00:00:00, 2000-01-05 00:00:00]
C    [2000-01-04 00:00:00, 2000-01-05 00:00:00]
dtype: object

In [25]:
def subtract_and_divide(x, sub, divide=1):
    return (x - sub) / divide
tsdf.apply(subtract_and_divide, args=(5,), divide=3) 
# args is pass to function sub == 5, or you can pass directly by using arg name

,A,B,C
2000-01-01,-0.333333,0.333333,0.000000
2000-01-02,1.000000,-0.333333,-0.333333
2000-01-03,-1.666667,-1.000000,0.666667
2000-01-04,0.333333,1.333333,2.000000
2000-01-05,0.000000,-1.333333,-1.000000
2000-01-06,-1.000000,0.000000,-0.666667
2000-01-07,-0.666667,1.000000,1.000000
2000-01-08,1.333333,1.666667,0.333333
2000-01-09,0.666667,-0.666667,1.666667
2000-01-10,-1.333333,0.666667,1.333333


In [26]:
df_.iloc[3:7] = np.nan;df_

,A,B,C
2000-01-01,0.0,1.0,2.0
2000-01-02,1.0,2.0,3.0
2000-01-03,2.0,3.0,4.0
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,7.0,8.0,9.0
2000-01-09,8.0,9.0,10.0
2000-01-10,9.0,10.0,11.0


In [27]:
df_.apply(pd.Series.interpolate)

,A,B,C
2000-01-01,0.0,1.0,2.0
2000-01-02,1.0,2.0,3.0
2000-01-03,2.0,3.0,4.0
2000-01-04,3.0,4.0,5.0
2000-01-05,4.0,5.0,6.0
2000-01-06,5.0,6.0,7.0
2000-01-07,6.0,7.0,8.0
2000-01-08,7.0,8.0,9.0
2000-01-09,8.0,9.0,10.0
2000-01-10,9.0,10.0,11.0


In [28]:
tsdf.agg({'A': ['mean', 'min'], 'B': 'sum'})

,A,B
mean,4.5,NaN
min,0.0,NaN
sum,NaN,55.0


In [29]:
from functools import partial
q_25 = partial(pd.Series.quantile, q=0.25)
q_25.__name__ = '25%'
tsdf.agg(['count', 'mean', 'std', 'min', q_25, 'median', 'max'])

,A,B,C
count,10.00000,10.00000,10.00000
mean,4.50000,5.50000,6.50000
std,3.02765,3.02765,3.02765
min,0.00000,1.00000,2.00000
25%,2.25000,3.25000,4.25000
median,4.50000,5.50000,6.50000
max,9.00000,10.00000,11.00000


In [30]:
dfr = pd.DataFrame(np.random.randn(10, 3), columns=['A', 'B', 'C'], index=pd.date_range('1/1/2000', periods=10))
dfr

,A,B,C
2000-01-01,-0.708765,-0.040000,-1.346324
2000-01-02,-2.504550,-0.541016,-0.239120
2000-01-03,-0.394760,1.092936,-1.980849
2000-01-04,0.071223,-0.291398,0.193353
2000-01-05,-1.362141,-2.186876,-0.430433
2000-01-06,1.866404,0.334364,1.555045
2000-01-07,0.997354,0.969383,0.509079
2000-01-08,0.602864,-2.767392,0.036787
2000-01-09,-0.201211,-0.492687,1.540064
2000-01-10,-1.857811,0.846398,0.702701


In [31]:
##transform series
dfr.apply(np.abs)
dfr.transform(np.abs)
dfr.transform('abs')
dfr.transform(lambda x: x.abs())
np.abs(tsdf)
tsdf.A.transform(np.abs)
tsdf.transform([np.abs, lambda x: x+1])

A                 B                 C         
           absolute <lambda> absolute <lambda> absolute <lambda>
2000-01-01        4        5        6        7        5        6
2000-01-02        8        9        4        5        4        5
2000-01-03        0        1        2        3        7        8
2000-01-04        6        7        9       10       11       12
2000-01-05        5        6        1        2        2        3
2000-01-06        2        3        5        6        3        4
2000-01-07        3        4        8        9        8        9
2000-01-08        9       10       10       11        6        7
2000-01-09        7        8        3        4       10       11
2000-01-10        1        2        7        8        9       10

In [32]:
tsdf.transform({'A': np.abs, 'B': lambda x: x+1})

,A,B
2000-01-01,4,7
2000-01-02,8,5
2000-01-03,0,3
2000-01-04,6,10
2000-01-05,5,2
2000-01-06,2,6
2000-01-07,3,9
2000-01-08,9,11
2000-01-09,7,4
2000-01-10,1,8


In [33]:
s = pd.Series(['six', 'seven', 'six', 'seven', 'six'],index=['a', 'b', 'c', 'd', 'e'])
t = pd.Series({'six' : 6., 'seven' : 7.})
s.map(t)

a    6.0
b    7.0
c    6.0
d    7.0
e    6.0
dtype: float64

In [34]:
#random sampleing
import random
list = random.sample(range(10), 10)
list.sort()
from random import SystemRandom #crypto-secure random
cryptorand = SystemRandom()
cryptorand.shuffle(list)
np.random.shuffle(list)

In [35]:
import pandas.util.testing as tm
panel = tm.makePanel(5)
f = lambda x: (x.T).T ##transpose items and major_axis -> transpose major_axis and minor_axis
result = panel.apply(f, axis = ['items','major_axis'])
result.loc[:,:,'ItemA'] #you can access ItemA for A to E data

AttributeError: module 'pandas.util.testing' has no attribute 'makePanel'

In [ ]:
#unlike apply, you don't specify axises, so transfpose works just only 2-D
result = pd.Panel(dict([ (ax, f(panel.loc[:,:,ax]))for ax in panel.minor_axis ]))

In [ ]:
rng = pd.date_range('1/3/2000', periods=8)
ts = pd.Series(np.random.randn(8), index=rng)
ts2 = ts[[0, 3, 6]]
ts2.reindex(ts.index, method='pad')
ts2.reindex(ts.index,).fillna(method='pad')
ts2.reindex(ts.index, method='ffill', limit=1)
#ts2.reindex(ts.index, method='ffill', tolerance='2 day')

In [ ]:
df
df.drop(['a', 'd'], axis=0) #is the same as
df.reindex(df.index.difference(['a', 'd']))

In [ ]:
df.rename(str.upper)

In [ ]:
df.rename(columns={'one': 'foo', 'two': 'bar'}, index={'a': 'apple', 'b': 'banana', 'd': 'durian'})

In [ ]:
s.rename("scalar-name")

In [ ]:
df = pd.DataFrame({'col1' : np.random.randn(3), 'col2' : np.random.randn(3)}, index=['a', 'b', 'c'])
#for col in df:
#    print(col)
for index, row in df.iterrows():
    print(index)
    print(row)

In [ ]:
for i in df.itertuples():
    print(i)

In [ ]:
for item, frame in panel.iteritems():
    print(item)
    print(frame)

In [ ]:
df2 = pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]})
df2.T # is the same as...

In [ ]:
pd.DataFrame(dict((idx,values) for idx, values in df2.iterrows()))

In [ ]:
dict((idx,values) for idx, values in df2.iterrows())

In [ ]:
#py dict constructor
#https://docs.python.org/2/tutorial/datastructures.html#dictionaries
dict([('sape', 4139), ('guido', 4127), ('jack', 4098)])
dict(sape=4139, guido=4127, jack=4098)
{x: x**2 for x in (2, 4, 6)}

In [ ]:
#for list item
for i, v in enumerate(['tic', 'tac', 'toe']):
    print(i,v)
#for dicrionary
knights = {'gallahad': 'the pure', 'robin': 'the brave'}
for k, v in knights.items(): ##iteritems() for py.2
    print (k, v)

In [ ]:
basket = ['apple', 'orange', 'apple', 'pear', 'orange', 'banana']
for f in sorted(set(basket)):
    print (f)
##!!set removed duplication
for i in reversed(range(1,10,2)):
    print (i)
##zippig and make tuples
questions = ['name', 'quest', 'favorite color']
answers = ['lancelot', 'the holy grail', 'blue']
for q, a in zip(questions, answers):
    print ('What is your {0}?  It is {1}.'.format(q, a))

In [ ]:
df1 = pd.DataFrame({'one':[2,1,1,1],'two':[1,3,2,4],'three':[5,4,3,2]})
df1.sort_values(by='two')
df1[['one', 'two', 'three']].sort_values(by=['one','two'])
s[2] = np.nan
s.sort_values()
s.sort_values(na_position='first')

In [ ]:
s = pd.Series(np.random.permutation(10))
s.nsmallest(3)
s.nlargest(3)

In [ ]:
df1 = pd.DataFrame({'one':[2,1,1,1],'two':[1,3,2,4],'three':[5,4,3,2]})
df1.columns = pd.MultiIndex.from_tuples([('a','one'),('a','two'),('b','three')])
df1.sort_values(by=('a','two'))

In [ ]:
df = pd.DataFrame([['2016-07-09', datetime.datetime(2016, 3, 2)]] * 2, dtype='O')
df.apply(pd.to_datetime)